# Data from crawling Airlines' Timelines

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np


In [0]:
data_path = '/content/drive/My Drive/BT4222/Data/twitter_data.csv'

In [0]:
data = pd.read_csv(data_path)

In [5]:
data.shape

(10896, 7)

In [6]:
data.head()

,Unnamed: 0,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,0,1246636468396789762,73287339,True,2020-04-05 03:11:10,"@elsadiella Hi Elsa, thanks for your DM. Let u...",1.246635e+18
1,1,1245925067692707840,73287339,True,2020-04-03 04:04:18,"@emmyoga Hi Emmanuel, our team is working hard...",1.245919e+18
2,2,1245502987239337988,73287339,True,2020-04-02 00:07:07,"@epulzakaria Hiya, we’d like to look into this...",1.245309e+18
3,3,1245490068736880642,73287339,True,2020-04-01 23:15:47,"@SzigyartoMarton Hi Marton, can you please sen...",1.245240e+18
4,4,1245485543229702145,73287339,True,2020-04-01 22:57:48,"@rich_consumer Hi Richard, we’d like to look i...",1.245181e+18


In [7]:
data.columns

Index(['Unnamed: 0', 'tweet_id', 'authod_id', 'inbound', 'created_at', 'text',
       'in_response_to_tweet_id'],
      dtype='object')

In [8]:
# dropping the Unnamed column
data = data.drop(columns=['Unnamed: 0'])

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1246636468396789762,73287339,True,2020-04-05 03:11:10,"@elsadiella Hi Elsa, thanks for your DM. Let u...",1.246635e+18
1,1245925067692707840,73287339,True,2020-04-03 04:04:18,"@emmyoga Hi Emmanuel, our team is working hard...",1.245919e+18
2,1245502987239337988,73287339,True,2020-04-02 00:07:07,"@epulzakaria Hiya, we’d like to look into this...",1.245309e+18
3,1245490068736880642,73287339,True,2020-04-01 23:15:47,"@SzigyartoMarton Hi Marton, can you please sen...",1.245240e+18
4,1245485543229702145,73287339,True,2020-04-01 22:57:48,"@rich_consumer Hi Richard, we’d like to look i...",1.245181e+18
...,...,...,...,...,...,...
10891,1237592337582149632,543147712,True,Wed Mar 11 04:13:01 +0000 2020,@AirAsiaSupport saya membeli 3 tiket jakarta-b...,NaN
10892,1236868326141227008,399896955,True,Mon Mar 09 04:16:03 +0000 2020,@AirAsiaSupport จองตั๋วเลือกวิธีการจ่ายผ่านธนา...,NaN
10893,1237182189348904961,323387559,True,Tue Mar 10 01:03:14 +0000 2020,"@AirAsiaSupport heeeeeey please check your dm,...",NaN
10894,1235507071111426050,57284154,True,Thu Mar 05 10:06:55 +0000 2020,"@AirAsiaSupport Ok submitted few days ago, can...",1.233692e+18


We need to first find all the consumer messages ('in_response_to_tweet_id' has NaN) and then from there find out how many we actually have to train on

In [9]:
customer = data[np.isnan(data['in_response_to_tweet_id'])]

customer.head()

,Unnamed: 0,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
141,141,1229948723393896453,73287339,True,2020-02-19 02:00:01,Jetstar Asia introduces 4-weekly services from...,NaN
304,304,1214455887164837890,73287339,True,2020-01-07 07:57:01,The Business Times in Singapore report how Jet...,NaN
415,415,1194551480692477952,73287339,True,2019-11-13 09:44:01,Jetstar Asia will fly three weekly direct serv...,NaN
429,429,1191930461527916545,73287339,True,2019-11-06 04:09:01,"Popular Jetstar Asia destination, Okinawa, bec...",NaN
503,503,1178580248549572608,73287339,True,2019-09-30 08:00:02,Travellers rank Jetstar Asia's customer servic...,NaN


In [10]:
customer.shape

(2341, 7)

# Collecting Tweets that contain the company's handles

In [0]:
import os
import tweepy
import pandas as pd

In [3]:
import getpass # prevents echo that is unhidden

# allow users to key in tokens here:
consumerkey = getpass.getpass("Consumer Key: ")
consumersecret = getpass.getpass("Consumer Secret: ")
accesstoken = getpass.getpass("Access Token: ")
accesstokensecret = getpass.getpass("Access Token Secret: ")

Consumer Key: ··········
Consumer Secret: ··········
Access Token: ··········
Access Token Secret: ··········


In [0]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumerkey, consumersecret)
auth.set_access_token(accesstoken,accesstokensecret)

api = tweepy.API(auth, wait_on_rate_limit=True) # twitter api has a rate limit, and adding this helps the code to obey this limit 
                                                # instead of throwing error

# without a cursor, to properly handle the Tweepy SearchResults object, i need this:
# api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True) 
# because the object returned to me is a rich python object, and doing this allows me to parse it as json

In [0]:
# Defining the None type
NoneType = type(None)

## Testing things out

In [0]:
# find the most recent rpp number of tweets that contain the company handle

test = api.search(q="@AirAsia", lang="en", count=500)
test.keys()

In [24]:
testdf = pd.DataFrame(test['statuses'])

testdf.head()

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
0,Sat Apr 11 18:00:54 +0000 2020,1249034705283891202,1249034705283891202,RT @CheauMei: Why we don't want credit account...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 2856670651, 'id_str': '2856670651', 'na...",None,None,None,None,{'created_at': 'Wed Apr 08 11:37:58 +0000 2020...,False,50,0,False,False,en,NaN,NaN,NaN,NaN
1,Sat Apr 11 17:59:37 +0000 2020,1249034384302387202,1249034384302387202,RT @AirAsia: A message from Tony.\n\n#InThisTo...,False,"{'hashtags': [{'text': 'InThisTogether', 'indi...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 2715079393, 'id_str': '2715079393', 'na...",None,None,None,None,{'created_at': 'Sat Apr 11 12:56:54 +0000 2020...,False,115,0,False,False,en,False,NaN,NaN,NaN
2,Sat Apr 11 17:58:40 +0000 2020,1249034142647570434,1249034142647570434,RT @AirAsia: A message from Tony.\n\n#InThisTo...,False,"{'hashtags': [{'text': 'InThisTogether', 'indi...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 893478918652649472, 'id_str': '89347891...",None,None,None,None,{'created_at': 'Sat Apr 11 12:56:54 +0000 2020...,False,115,0,False,False,en,False,NaN,NaN,NaN
3,Sat Apr 11 17:57:49 +0000 2020,1249033931522846721,1249033931522846721,RT @CheauMei: Why we don't want credit account...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 1662709808, 'id_str': '1662709808', 'na...",None,None,None,None,{'created_at': 'Wed Apr 08 11:37:58 +0000 2020...,False,50,0,False,False,en,NaN,NaN,NaN,NaN
4,Sat Apr 11 17:57:46 +0000 2020,1249033916972797955,1249033916972797955,@narendramodi @DrSJaishankar @PMOIndia we obey...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,18839785.0,18839785,narendramodi,"{'id': 1233973117959032832, 'id_str': '1233973...",None,None,None,None,NaN,True,1,0,False,False,en,False,1.249030e+18,1249030045374361600,{'created_at': 'Sat Apr 11 17:42:23 +0000 2020...


In [25]:
testdf.shape

(100, 29)

## Airasia

In [0]:
handle = "AirAsia"

In [0]:
tweet_id = []
authod_id = []
inbound = [] # inbound are tweets that arent in reply to anything
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
for t in tweepy.Cursor(api.search, q="@"+handle, count=500, lang='en').items():
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(type(t.in_reply_to_user_id_str) == NoneType)
  created_at.append(t.created_at)
  text.append(t.text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [59]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

2676
2676
2676
2676
2676
2676


In [60]:
aa_main = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

aa_main.shape

(2676, 6)

In [61]:
aa_main.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1249226751487561728,318555989,True,2020-04-12 06:44:01,RT @abasirhb: @staronline What's the point if ...,None
1,1249226229946830848,144476455,True,2020-04-12 06:41:57,RT @_hafiz: @kanjiro1922 @weechookeong @AirAsi...,None
2,1249226107607375872,696873464443990016,False,2020-04-12 06:41:28,@AirAsia WITH CURRENT SITUATION AA SHOULD HAVE...,1226071184992657408
3,1249225612423622659,2815806303,False,2020-04-12 06:39:30,@EaseMyTrip @AirAsia when will you reschedule ...,None
4,1249225056862875649,891987654333661184,False,2020-04-12 06:37:17,@DelhiAirport @flyspicejet @AirAsiaIndian @goa...,1248501428374495233


## AirAsia Support

In [0]:
handle = "AirAsiaSupport"

In [0]:
tweet_id = []
authod_id = []
inbound = [] # inbound are tweets that arent in reply to anything
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
for t in tweepy.Cursor(api.search, q="@"+handle, count=500, lang='en').items():
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(type(t.in_reply_to_user_id_str) == NoneType)
  created_at.append(t.created_at)
  text.append(t.text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [65]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

3605
3605
3605
3605
3605
3605


In [66]:
aa_svc = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

aa_svc.shape

(3605, 6)

In [67]:
aa_svc.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id
0,1249226772790435840,747571081972637696,False,2020-04-12 06:44:06,It’s been a month and this is still not resolv...,1238457594114478085
1,1249224721888993283,1236202018529656837,False,2020-04-12 06:35:57,@AirAsiaSupport The way AirAsia is dealing wit...,1246614261834461184
2,1249223468958109696,161345240,True,2020-04-12 06:30:59,"RT @irisyen6: @malaysiakini Hey Malaysiakini, ...",None
3,1249223204876333064,254467028,True,2020-04-12 06:29:56,"RT @irisyen6: @malaysiakini Hey Malaysiakini, ...",None
4,1249222671113441281,563475279,False,2020-04-12 06:27:48,@AirAsiaSupport please please reply on my dm,1225976008118898688


In [68]:
## Need to combine both datasets from airasia together
aa_all = pd.concat([aa_main, aa_svc])

aa_all.shape

(6281, 6)

In [69]:
# make sure to make these tweets as customer tweets directed at airasia
aa_all['customers_of'] = 'AirAsia'

aa_all.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id,customers_of
0,1249226751487561728,318555989,True,2020-04-12 06:44:01,RT @abasirhb: @staronline What's the point if ...,None,AirAsia
1,1249226229946830848,144476455,True,2020-04-12 06:41:57,RT @_hafiz: @kanjiro1922 @weechookeong @AirAsi...,None,AirAsia
2,1249226107607375872,696873464443990016,False,2020-04-12 06:41:28,@AirAsia WITH CURRENT SITUATION AA SHOULD HAVE...,1226071184992657408,AirAsia
3,1249225612423622659,2815806303,False,2020-04-12 06:39:30,@EaseMyTrip @AirAsia when will you reschedule ...,None,AirAsia
4,1249225056862875649,891987654333661184,False,2020-04-12 06:37:17,@DelhiAirport @flyspicejet @AirAsiaIndian @goa...,1248501428374495233,AirAsia


## JetStar Asia

In [0]:
handle = "JetStar_Asia"

In [0]:
tweet_id = []
authod_id = []
inbound = [] # inbound are tweets that arent in reply to anything
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
for t in tweepy.Cursor(api.search, q="@"+handle, count=500, lang='en').items():
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(type(t.in_reply_to_user_id_str) == NoneType)
  created_at.append(t.created_at)
  text.append(t.text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [73]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

31
31
31
31
31
31


In [74]:
jetstar = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

jetstar.shape

(31, 6)

In [75]:
jetstar['customers_of'] = "JetStar Asia"
jetstar.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id,customers_of
0,1249193154512916480,375541562,False,2020-04-12 04:30:31,@Jetstar_Asia Please check your DM. Thanks @Je...,1249190016930271232,JetStar Asia
1,1249189062759665667,375541562,False,2020-04-12 04:14:16,@Jetstar_Asia Pleas help me. I have also tried...,1249186246884196357,JetStar Asia
2,1249040470401060864,1024338385404682246,True,2020-04-11 18:23:48,RT @monilsj: Hi @Jetstar_Asia! I have noticed ...,None,JetStar Asia
3,1249038696214458369,286610770,True,2020-04-11 18:16:45,Hi @Jetstar_Asia! I have noticed that Air Asia...,None,JetStar Asia
4,1248566696798867456,1048797604019036160,True,2020-04-10 11:01:12,RT @RayMSLim: @Jetstar_Asia why do you make it...,None,JetStar Asia


# Scoot

In [0]:
handle = "@flyscoot"

In [0]:
tweet_id = []
authod_id = []
inbound = [] # inbound are tweets that arent in reply to anything
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
for t in tweepy.Cursor(api.search, q="@"+handle, count=500, lang='en').items():
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(type(t.in_reply_to_user_id_str) == NoneType)
  created_at.append(t.created_at)
  text.append(t.text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [79]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

83
83
83
83
83
83


In [80]:
scoot = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

scoot.shape

(83, 6)

In [81]:
scoot['customers_of'] = "Scoot"
scoot.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id,customers_of
0,1249224150935375872,2951505446,False,2020-04-12 06:33:41,@flyscoot How If i booked ticket thru travel A...,1239768455642894338,Scoot
1,1249219496566345728,221646790,False,2020-04-12 06:15:12,@AirAsia vs @flyscoot \n\nAA just refused to b...,None,Scoot
2,1249170831202299905,1173705523335585793,False,2020-04-12 03:01:49,@flyscoot \n29 days ago you cancelled my fligh...,None,Scoot
3,1248979782303440896,34683910,False,2020-04-11 14:22:39,@flyscoot \nWe have received your request 2002...,None,Scoot
4,1248979537989427202,34683910,False,2020-04-11 14:21:41,@flyscoot why you don’t answer mail from me,None,Scoot


# Lion Airways

In [0]:
handle = "LionAirways"

In [0]:
tweet_id = []
authod_id = []
inbound = [] # inbound are tweets that arent in reply to anything
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
for t in tweepy.Cursor(api.search, q="@"+handle, count=500, lang='en').items():
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(type(t.in_reply_to_user_id_str) == NoneType)
  created_at.append(t.created_at)
  text.append(t.text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [86]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

3
3
3
3
3
3


In [87]:
lionair = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

lionair.shape

(3, 6)

In [88]:
lionair['customers_of'] = "Lion Air"
lionair.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id,customers_of
0,1248647854396256256,52436759,False,2020-04-10 16:23:41,@Linny57284055 @HailMarxism @Tony30DC @premnsi...,1248591312510529536,Lion Air
1,1248591312510529536,1440864086,False,2020-04-10 12:39:01,@HailMarxism @keithpp @Tony30DC @premnsikka @e...,1248466519371042816,Lion Air
2,1248251851012812800,2238052741,False,2020-04-09 14:10:07,@ThaiLionairJP @Paytm @Paytmcare I received b...,None,Lion Air


# Cebu Pacific Air

In [0]:
handle = "CebuPacificAir"

In [0]:
tweet_id = []
authod_id = []
inbound = [] # inbound are tweets that arent in reply to anything
created_at = []
text = []
reponses_tweet_id = []
in_response_to_tweet_id = []

In [0]:
for t in tweepy.Cursor(api.search, q="@"+handle, count=500, lang='en').items():
  tweet_id.append(t.id_str)
  authod_id.append(t.user.id_str)
  inbound.append(type(t.in_reply_to_user_id_str) == NoneType)
  created_at.append(t.created_at)
  text.append(t.text)
  # responses_tweet_id.append(t.reply_count)
  in_response_to_tweet_id.append(t.in_reply_to_status_id_str)

In [92]:
# confirming all same length
for i in (tweet_id, authod_id, inbound, created_at, text, in_response_to_tweet_id):
  print(len(i))

1245
1245
1245
1245
1245
1245


In [93]:
cebu = pd.DataFrame(data={'tweet_id': tweet_id, 'authod_id': authod_id, 'inbound': inbound, 'created_at': created_at, 'text': text, 'in_response_to_tweet_id': in_response_to_tweet_id})

cebu.shape

(1245, 6)

In [94]:
cebu['customers_of'] = "Cebu Pacific Air"
cebu.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id,customers_of
0,1249222128118804481,782755174666297344,False,2020-04-12 06:25:39,@CebuPacificAir Hi Cebu Pac. We have a flight ...,1247743209867898881,Cebu Pacific Air
1,1249216963504517120,1232551202920886272,True,2020-04-12 06:05:08,RT @CebuPacificAir: Cebu Pacific Advisory \nA...,None,Cebu Pacific Air
2,1249204499236995072,1087908731386355712,False,2020-04-12 05:15:36,@CebuPacificAir travel fund option is still no...,None,Cebu Pacific Air
3,1249202059053797378,183077675,False,2020-04-12 05:05:54,"@CebuPacificAir Hi! Re travel fund, can it be ...",None,Cebu Pacific Air
4,1249194309154492418,939041621575745536,False,2020-04-12 04:35:06,@CebuPacificAir where can I find my reference ...,None,Cebu Pacific Air


# Combining the customer tweets from both

In [95]:
cust_tweets = pd.concat([jetstar, aa_all])
cust_tweets = pd.concat([cust_tweets, scoot])
cust_tweets = pd.concat([cust_tweets, lionair])
cust_tweets = pd.concat([cust_tweets, cebu])
cust_tweets.shape

(7643, 7)

In [96]:
cust_tweets.head()

,tweet_id,authod_id,inbound,created_at,text,in_response_to_tweet_id,customers_of
0,1249193154512916480,375541562,False,2020-04-12 04:30:31,@Jetstar_Asia Please check your DM. Thanks @Je...,1249190016930271232,JetStar Asia
1,1249189062759665667,375541562,False,2020-04-12 04:14:16,@Jetstar_Asia Pleas help me. I have also tried...,1249186246884196357,JetStar Asia
2,1249040470401060864,1024338385404682246,True,2020-04-11 18:23:48,RT @monilsj: Hi @Jetstar_Asia! I have noticed ...,None,JetStar Asia
3,1249038696214458369,286610770,True,2020-04-11 18:16:45,Hi @Jetstar_Asia! I have noticed that Air Asia...,None,JetStar Asia
4,1248566696798867456,1048797604019036160,True,2020-04-10 11:01:12,RT @RayMSLim: @Jetstar_Asia why do you make it...,None,JetStar Asia


In [100]:
# making sure the disparity exists between inbound and those with None in that col
test_in = cust_tweets["in_response_to_tweet_id"].apply(lambda x: type(x) == NoneType)
test_in.value_counts(normalize=True)

True     0.57883
False    0.42117
Name: in_response_to_tweet_id, dtype: float64

In [101]:
cust_tweets["inbound"].value_counts(normalize=True)

False    0.778752
True     0.221248
Name: inbound, dtype: float64

In [102]:
# replace the inbound column with test_in
cust_tweets["inbound"] = test_in

cust_tweets["inbound"].value_counts(normalize=True)

True     0.57883
False    0.42117
Name: inbound, dtype: float64

In [0]:
from google.colab import files

# to download the csv
cust_tweets.to_csv('cust_tweets.csv')
files.download('cust_tweets.csv')

# Debugging

In [97]:
# inbound only
test = cust_tweets.loc[cust_tweets["inbound"] == True]
test.shape

(1691, 7)

In [98]:
count = 0
for i in cust_tweets["in_response_to_tweet_id"]:
  if type(i) == NoneType:
    count += 1
  
print(count)

4424


In [0]:
# Since None is a singleton, we need this to test whether another variable shares the None type
# reference: https://stackoverflow.com/questions/23086383/how-to-test-nonetype-in-python
NoneType = type(None)

In [55]:
for i in scoot["in_response_to_tweet_id"].iloc[:6]:
  print(type(i) == NoneType)

True
True
True
True
False
True
